<a href="https://colab.research.google.com/github/Sourav1429/Restricted_BNN/blob/main/FFNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import tensorflow as tf

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import tensorflow_probability as tfp
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the data
data = pd.read_excel('/content/Pulsar_reduced.xlsx')

# Assuming 'target' is the column containing the labels
X = data.drop('label', axis=1)
y = data['label']

# Handling missing values by filling with the mean
X = X.fillna(X.mean())

# Split the data into training, testing, and validation sets
# 80% for training, 10% for validation, and 10% for testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(X_validation)
X_test_scaled = scaler.transform(X_test)

# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential([
        tfp.layers.DistributionLambda(
            lambda t: tfp.distributions.MultivariateNormalDiag(
                loc=tf.zeros(n), scale_diag=tf.ones(n)
            )
        )
    ])
    return prior_model

# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential([
        tfp.layers.VariableLayer(
            tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
        ),
        tfp.layers.MultivariateNormalTriL(n),
    ])
    return posterior_model

def create_ffnn_model(train_size):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=X_train_scaled.shape[1]))

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in [2,2]:  #  number of units in hidden layers
        model.add(tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation='relu',
        ))

    model.add(layers.Dense(units=1, activation='sigmoid'))
    return model

ffnn_model = create_ffnn_model(train_size=len(X_train_scaled))

#  model with the Adam optimizer and binary cross-entropy loss
ffnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# training history
history = ffnn_model.fit(X_train_scaled, y_train, validation_data=(X_validation_scaled, y_validation),
                         epochs=10, batch_size=32, verbose=0)


In [18]:
df = pd.DataFrame({'FFNN':np.array(history.history['loss'])})
df.to_excel('FFNN_pulsar.xlsx')

In [19]:
print("Accuracies:",history.history['accuracy'])

Accuracies: [0.6078627109527588, 0.8058272004127502, 0.8715825080871582, 0.9046098589897156, 0.9050089716911316, 0.9076032638549805, 0.9076032638549805, 0.9076032638549805, 0.9076032638549805, 0.9076032638549805]
